In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import cv2
import numpy as np
import pandas as pd
#import tensorflow as tf
  
import gzip
import pickle
import csv
from scipy.spatial import distance as dist
import imutils

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


train_images = pd.read_pickle('/content/drive/My Drive/Colab/train_max_x')
test_images = pd.read_pickle('/content/drive/My Drive/Colab/test_max_x')
train_labels = pd.read_csv("/content/drive/My Drive/Colab/train_max_y.csv")

In [4]:
import cv2
import numpy as np
import sys
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils
from pandas import DataFrame
from scipy import ndimage
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
from tensorflow import keras
from keras.models import Sequential
from keras import datasets, layers, models

# train_images = pd.read_pickle('train_max_x')
# test_images = pd.read_pickle('test_max_x')
# train_labels = pd.read_csv("train_max_y.csv")
# train_labels.drop(['Id'], inplace=True, axis=1)
# train_labels = train_labels.pop('Label')

retval, train_threshold = cv2.threshold(train_images, 250, maxval=255, type=cv2.THRESH_BINARY_INV)
retval_test, test_threshold = cv2.threshold(test_images, 250, maxval=255, type=cv2.THRESH_BINARY_INV)

x_train = train_threshold
y_train = train_labels
train_images = test_threshold

x_train, x_test, y_train, y_test = train_test_split(train_threshold, train_labels, test_size=0.25, random_state=33)

x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')

numClass = 10
datasize = 128

x_train = np.divide(x_train, 255.0)
x_train = x_train[:,:, :, np.newaxis]
x_test = np.divide(x_test, 255.0)
x_test = x_test[:, :, :, np.newaxis]
y_train = np.array(y_train)
y_test = np.array(y_test)

x_train = keras.utils.to_categorical(x_train, numClass)
x_test = keras.utils.to_categorical(x_test, numClass)
y_train = keras.utils.to_categorical(y_train, numClass)
y_test = keras.utils.to_categorical(y_test, numClass)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(datasize,datasize,10)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(numClass))

model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

batch_size = 128
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,validation_data=(x_test, y_test))

#predicted = model.predict(test_images, verbose=0)


Using TensorFlow backend.
